In [1]:
!gcloud config get-value account

815755318672-compute@developer.gserviceaccount.com


In [2]:
import pandas as pd
import config
from config import PROJECT_ID, LOCATION, MEMORY_ENGINE_DISPLAY_NAME, INPUT_FILE_PATH,MAIN_AGENT,CRITIC_MODEL,CODE_MODEL, API_TYPE_GEMINI,API_TYPE_SONNET,QUICK_SMART_MODEL,ANTHROPIC_API_KEY,basic_config_agent
from dynamic_graph.outputsModels import *
from dynamic_graph.main_graph_functions import run_collaborative_planning
from prompts import PromptEngine
from dynamic_graph.state import WorkflowState
from dynamic_graph.main_graph_functions import build_and_run_graph

In [3]:
# # --- Konfiguracja czatu grupowego ---
# main_agent_configuration={"cache_seed": 42,"seed": 42,"temperature": 0.0,
#                         "config_list": basic_config_agent(agent_name=MAIN_AGENT, api_type=API_TYPE_GEMINI, location=LOCATION, project_id=PROJECT_ID)}
# critic_agent_configuration ={"cache_seed": 42,"seed": 42,"temperature": 0.0,
#                         "config_list": basic_config_agent(api_key=ANTHROPIC_API_KEY,agent_name=CRITIC_MODEL, api_type=API_TYPE_SONNET)}
router_agent_configuration = {"cache_seed": 42,"seed": 42,"temperature": 0.0,
                              "config_list": basic_config_agent(agent_name=QUICK_SMART_MODEL, api_type=API_TYPE_GEMINI, location=LOCATION, project_id=PROJECT_ID)}


In [4]:
if __name__ == '__main__':
    # Konfiguracja dla lekkiego i szybkiego modelu Rutera
    
    
    # Wysokopoziomowa misja od użytkownika
    mission= """
    Zaprojektuj ODPORNY NA BŁĘDY, sekwencyjny przepływ pracy do weryfikacji modelu przyczynowego.
    Kroki to: load_data, discover_causality i validate_model.
    Węzeł 'discover_causality' jest ryzykowny i po nim musi nastąpić sprawdzenie,
    czy operacja się powiodła. W razie błędu, proces musi zostać skierowany do uniwersalnego debuggera,
    a następnie spróbować ponownie wykonać krok 'discover_causality'.
    """
    
    # Uruchom całą fazę planowania jedną komendą
    final_plan = run_collaborative_planning(mission, router_agent_configuration)
    
    if final_plan:
        print("\n" + "="*50)
        print("### OTRZYMANY PLAN GOTOWY DO WYKONANIA ###")
        print(final_plan.model_dump_json(indent=2, by_alias=True))
        
        initial_state = {
            "mission":mission,
            "input_path":INPUT_FILE_PATH, # Używamy ścieżki z pliku konfiguracyjnego
            "dataframe":pd.DataFrame(), # Pusta ramka danych na start
            "causal_graph":None,
            "validation_report":{},
            "final_summary":"",
            "error":""
        }
        
        # Uruchamiamy fabrykę, aby zbudowała i uruchomiła graf na podstawie planu
        build_and_run_graph(
            plan=final_plan,
            state_schema=WorkflowState,
            initial_state=initial_state
        )
        
    else:
        print("\nNie udało się wypracować planu. Sprawdź logi w `reports/`.")

    
    


### ROZPOCZYNANIE FAZY PROJEKTOWEJ ###
--- RUTER AGENTÓW: Uruchamiam uniwersalny wybór zespołu... ---
INFO [StrucRes]: Próba 1/3...

>>>>>>>> USING AUTO REPLY...
INFO [StrucRes]: Odpowiedź poprawna i zwalidowana.
--- RUTER AGENTÓW: Wybrany zespół -> {'thought_process': "Misja wymaga zaprojektowania ODPORNEGO NA BŁĘDY, sekwencyjnego przepływu pracy do weryfikacji modelu przyczynowego, ze szczególnym uwzględnieniem obsługi błędów i ponownych prób dla kroku 'discover_causality'.\n\n1.  **Wybór Krytyka:** Dostępny jest tylko jeden ekspert wyraźnie pełniący rolę krytyka: 'Krytyk_Jakosci' (AEGIS, THE UNYIELDING GUARDIAN OF QUALITY). Jest to oczywisty wybór.\n\n2.  **Wybór Planistów (1-2):**\n    *   **Ekspert_Przyczynowosci (CAUSAL MODELING ARCHITECT):** Jest kluczowy dla rdzenia misji, czyli 'discover_causality' i 'validate_model'. Bez tego eksperta, model przyczynowy nie mógłby być prawidłowo zaprojektowany ani zweryfikowany.\n    *   **Architekt_GPT (ADVANCED WORKFLOW ARCHITECT (GPT-Powe